In [1]:
# Relevant module imports and installs
import pandas as pd
!pip install pulp brotli fuzzywuzzy
import pulp as plp
import sys 
import os
from collections import defaultdict
from fuzzywuzzy import process
import time


[notice] A new release of pip is available: 23.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
solve_season = '2024-25'
solve_gameweek = 13
load_projections_from_file = False

In [3]:
# Get the absolute path to the directory containing the Python file
module_path = os.path.abspath(os.path.join('..', '..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

# Now you can import the module
from projections import generate_projections, generate_stat_projections, append_stat_projections

if not load_projections_from_file:
    point_projections = generate_projections()
    stat_projections = generate_stat_projections()

    projections_data = append_stat_projections(point_projections, stat_projections, solve_gameweek)
    projections_data.to_csv('gameweek_projections.csv', index=False)
    print('Generated new projections.')
else:
    try:
        projections_data = pd.read_csv('gameweek_projections.csv')
        print('Loaded projections from CSV file.')
    except FileNotFoundError:
        print('Tried to load from CSV file, but it does not exist. Generating new projections...')
        point_projections = generate_projections()
        stat_projections = generate_stat_projections()
        projections_data = append_stat_projections(point_projections, stat_projections, solve_gameweek)

Generated new projections.


### Player Manipulation

### Player Force/Banning

In [4]:
def fuzzy_ban_players(df, ban_ids):
    while True:
        search_name = input("Enter player name to ban (or press enter to finish): ").strip()
        
        if search_name.lower() == '':
            break
        
        # Perform fuzzy matching with a lower score cutoff and no limit
        matches = process.extractBests(search_name, df['Name'].tolist(), score_cutoff=50, limit=10)
        
        if not matches:
            print("No matches found. Please try again.")
            continue
        
        # Display matches
        print("Matches found:")
        for idx, (name, score) in enumerate(matches, 1):
            player_index = df[df['Name'] == name].index[0]
            player_id = df.loc[player_index, 'ID']
            print(f"{idx}. {name} (ID: {player_id}, Index: {player_index}, Score: {score})")
        
        # Ask user to select a match
        while True:
            choice = input("Enter the number of the player to ban (or 'skip' to search again): ")
            if choice.lower() == 'skip':
                break
            try:
                choice_idx = int(choice) - 1
                if 0 <= choice_idx < len(matches):
                    selected_name = matches[choice_idx][0]
                    selected_index = df[df['Name'] == selected_name].index[0]
                    selected_id = df.loc[selected_index, 'ID']
                    ban_ids.append(selected_index)
                    print(f"Banned: {selected_name} (ID: {selected_id}, Index: {selected_index})")
                    break
                else:
                    print("Invalid choice. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a number or 'skip'.")
    
    return ban_ids

ban_ids = []
ban_ids = fuzzy_ban_players(projections_data, ban_ids)
print("Final ban list (indices):", ban_ids)

Final ban list (indices): []


In [5]:
def fuzzy_force_players(df, force_ids):
    while True:
        search_name = input("Enter player name to force (or press enter to finish): ").strip()
        
        if search_name.lower() == '':
            break
        
        # Perform fuzzy matching with a lower score cutoff and no limit
        matches = process.extractBests(search_name, df['Name'].tolist(), score_cutoff=50, limit=10)
        
        if not matches:
            print("No matches found. Please try again.")
            continue
        
        # Display matches
        print("Matches found:")
        for idx, (name, score) in enumerate(matches, 1):
            player_index = df[df['Name'] == name].index[0]
            player_id = df.loc[player_index, 'ID']
            print(f"{idx}. {name} (ID: {player_id}, Index: {player_index}, Score: {score})")
        
        # Ask user to select a match
        while True:
            choice = input("Enter the number of the player to force (or 'skip' to search again): ")
            if choice.lower() == 'skip':
                break
            try:
                choice_idx = int(choice) - 1
                if 0 <= choice_idx < len(matches):
                    selected_name = matches[choice_idx][0]
                    selected_index = df[df['Name'] == selected_name].index[0]
                    selected_id = df.loc[selected_index, 'ID']
                    force_ids.append(selected_index)  # Add to force_ids instead of ban_ids
                    print(f"Forced: {selected_name} (ID: {selected_id}, Index: {selected_index})")
                    break
                else:
                    print("Invalid choice. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a number or 'skip'.")
    
    return force_ids

force_ids = []
force_ids = fuzzy_force_players(projections_data, force_ids)
print("Final force list (indices):", force_ids)

Final force list (indices): []


# 2024/25 GW13 Challenge: Haaland or Salah? - Liverpool and Man City players earn double points

In [6]:
# Double points for Liverpool and Man City players
double_points_teams = ['Liverpool', 'Man City']
projections_data.loc[projections_data['Team'].isin(double_points_teams), 'Predicted_Points'] *= 2

### Optimisation

In [9]:
# Get the number of players and their list of ids
player_ids = projections_data['ID'].tolist()
player_count = len(player_ids)

# Set up the problem
model = plp.LpProblem("fpl-challenge", plp.LpMaximize)

# Define the decision variables
lineup = [
    plp.LpVariable(f"lineup_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Define captain variables
captain = [
    plp.LpVariable(f"captain_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Set the objective function (the number of points scored by the team, with captain's points doubled)
model += plp.lpSum([lineup[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)]) + \
         plp.lpSum([captain[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)])

# Constraints

# Total number of players = 5
model += plp.lpSum(lineup) == 5

# List players by index to be EXCLUDED from the lineup
for id in ban_ids:
    model += lineup[id] == 0

for id in force_ids:
    model += lineup[id] == 1

# Exactly one captain
model += plp.lpSum(captain) == 1

# Captain must be in the lineup
for i in range(player_count):
    model += captain[i] <= lineup[i]

# Exactly 1 Goalkeeper
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Goalkeeper']) == 1

# At least 1 Defender
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Defender']) >= 1

# At least 1 Midfielder
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Midfielder']) >= 1

# At least 1 Forward
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Forward']) >= 1

# Budget constraint
model += plp.lpSum([lineup[i] * projections_data.loc[i, 'Cost'] for i in range(player_count)]) <= 44

# Solve the problem
model.solve()

# Function to print players by position
def print_players_by_position(players_dict):
    total_points = 0
    total_cost = 0
    for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
        if position in players_dict:
            print(f"\n{position}:")
            for player in players_dict[position]:
                captain_str = " (C)" if player['Captain'] else ""
                points = player['Predicted_Points'] * (2 if player['Captain'] else 1)
                print(f"  {player['Name']}{captain_str} - {player['Team']} - Cost: {player['Cost']}m - Predicted Points: {points}")
                total_points += points
                total_cost += player['Cost']
    print(f"\nTotal Predicted Points: {round(total_points, 2)}")
    print(f"Total Cost: {round(total_cost, 2)}m")

# Print the results
print("Status:", plp.LpStatus[model.status])
selected_players = defaultdict(list)
for i in range(player_count):
    if lineup[i].value() == 1:
        player = projections_data.loc[i]
        selected_players[player['Position']].append({
            'Name': player['Name'],
            'Team': player['Team'],
            'Cost': player['Cost'],
            'Predicted_Points': player['Predicted_Points'],
            'Captain': captain[i].value() == 1
        })

def solution_to_txt(players_dict, filename="solution.txt", encoding="utf-8"):
    total_points = 0
    total_cost = 0
    with open(filename, 'w', encoding=encoding) as f:
        f.write(f'Current Date & Time: {time.strftime("%Y-%m-%d")} - {time.strftime("%H:%M:%S")}\n')
        for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
            if position in players_dict:
                f.write(f"\n{position}:\n")
                for player in players_dict[position]:
                    captain_str = " (C)" if player['Captain'] else ""
                    points = player['Predicted_Points'] * (2 if player['Captain'] else 1)
                    f.write(f"  {player['Name']}{captain_str} - {player['Team']} - Cost: {player['Cost']}m - Predicted Points: {points}\n")
                    total_points += points
                    total_cost += player['Cost']
        f.write(f"\nTotal Predicted Points: {round(total_points, 2)}\n")
        f.write(f"Total Cost: {round(total_cost, 2)}m\n")

solution_to_txt(selected_players, "optimal_solution.txt")
print_solution = False
if print_solution:
    print("\nOptimal Lineup:")
    print_players_by_position(selected_players)

Status: Optimal

Optimal Lineup:

Goalkeeper:
  Ederson M. - Man City - Cost: 5.5m - Predicted Points: 6.88

Defender:
  Walker - Man City - Cost: 5.2m - Predicted Points: 6.22

Midfielder:
  Gravenberch - Liverpool - Cost: 5.0m - Predicted Points: 6.8
  M.Salah (C) - Liverpool - Cost: 13.1m - Predicted Points: 29.84

Forward:
  Haaland - Man City - Cost: 15.1m - Predicted Points: 12.58

Total Predicted Points: 62.32
Total Cost: 43.9m


In [10]:
def print_top_scorers_by_position(projections_data):
    for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
        top_scorers = projections_data[projections_data['Position'] == position].nlargest(10, 'Predicted_Points')
        print(f'Top {position}s: ')
        display(top_scorers)

if print_solution:
    print_top_scorers_by_position(projections_data)

Top Goalkeepers: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
195,347,Ederson M.,Man City,30,Goalkeeper,5.5,6.88,90,Liverpool (A),0.0,0.0,0.0,0.200,0.0
247,443,Sels,Nott'm Forest,21,Goalkeeper,4.7,4.31,90,Ipswich (H),0.0,0.0,0.0,0.444,0.0
55,91,Flekken,Brentford,152,Goalkeeper,4.5,4.06,90,Leicester (H),0.0,0.0,0.0,0.364,0.0
83,146,Verbruggen,Brighton,152,Goalkeeper,4.5,4.03,90,Southampton (H),0.0,0.0,0.0,0.400,0.0
281,508,Vicario,Spurs,106,Goalkeeper,5.0,4.03,90,Fulham (H),0.0,0.0,0.0,0.308,0.0
9,15,Raya,Arsenal,200,Goalkeeper,5.5,4.01,90,West Ham (A),0.0,0.0,0.0,0.444,0.0
215,383,Onana,Man Utd,38,Goalkeeper,5.1,3.95,90,Everton (H),0.0,0.0,0.0,0.454,0.0
100,185,Sánchez,Chelsea,200,Goalkeeper,4.7,3.90,90,Aston Villa (H),0.0,0.0,0.0,0.308,0.0
107,201,Henderson,Crystal Palace,241,Goalkeeper,4.4,3.63,90,Newcastle (H),0.0,0.0,0.0,0.250,0.0
315,568,Muric,Ipswich,286,Goalkeeper,4.4,3.57,90,Nott'm Forest (A),0.0,0.0,0.0,0.200,0.0


Top Defenders: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
189,339,Virgil,Liverpool,152,Defender,6.3,7.74,90,Man City (H),0.091,0.077,0.161,0.267,0.103
178,326,Konaté,Liverpool,73,Defender,5.4,7.36,90,Man City (H),0.062,0.046,0.106,0.267,0.071
197,350,Gvardiol,Man City,97,Defender,6.3,7.26,90,Liverpool (A),0.076,0.101,0.170,0.200,0.086
185,335,Robertson,Liverpool,243,Defender,5.9,6.36,61,Man City (H),0.062,0.117,0.171,0.267,0.071
205,363,Walker,Man City,241,Defender,5.2,6.22,83,Liverpool (A),0.042,0.109,0.146,0.200,0.049
190,340,Akanji,Man City,207,Defender,5.4,4.76,66,Liverpool (A),0.051,0.062,0.109,0.200,0.059
200,355,Lewis,Man City,241,Defender,4.8,4.76,65,Liverpool (A),0.048,0.098,0.141,0.200,0.056
18,26,Alex Moreno,Nott'm Forest,200,Defender,4.4,4.68,82,Ipswich (H),0.077,0.031,0.105,0.422,0.087
319,573,Milenković,Nott'm Forest,190,Defender,4.5,4.52,90,Ipswich (H),0.077,0.046,0.119,0.444,0.087
275,495,Pedro Porro,Spurs,200,Defender,5.5,4.48,90,Fulham (H),0.165,0.162,0.300,0.308,0.191


Top Midfielders: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
180,328,M.Salah,Liverpool,63,Midfielder,13.1,14.92,90,Man City (H),0.372,0.340,0.585,0.267,0.492
196,348,Foden,Man City,241,Midfielder,9.2,9.82,82,Liverpool (A),0.258,0.189,0.398,0.200,0.313
345,625,Gündogan,Man City,80,Midfielder,6.5,8.04,90,Liverpool (A),0.115,0.181,0.275,0.200,0.127
193,345,De Bruyne,Man City,21,Midfielder,9.4,7.84,59,Liverpool (A),0.000,0.000,0.000,0.000,0.000
98,182,Palmer,Chelsea,241,Midfielder,10.9,7.19,90,Aston Villa (H),0.439,0.297,0.606,0.308,0.615
192,342,Bernardo,Man City,173,Midfielder,6.4,7.00,72,Liverpool (A),0.133,0.233,0.336,0.200,0.150
175,323,Gravenberch,Liverpool,152,Midfielder,5.0,6.80,90,Man City (H),0.111,0.158,0.252,0.267,0.123
11,17,Saka,Arsenal,241,Midfielder,10.2,6.78,88,West Ham (A),0.331,0.244,0.495,0.444,0.432
278,503,Son,Spurs,114,Midfielder,9.9,6.43,86,Fulham (H),0.386,0.136,0.470,0.308,0.536
186,336,Szoboszlai,Liverpool,98,Midfielder,6.3,5.74,59,Man City (H),0.150,0.094,0.230,0.267,0.174


Top Forwards: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
198,351,Haaland,Man City,161,Forward,15.1,12.58,90,Liverpool (A),0.412,0.183,0.519,0.200,0.557
249,447,Wood,Nott'm Forest,154,Forward,6.6,6.73,83,Ipswich (H),0.356,0.034,0.378,0.444,0.476
172,316,Darwin,Liverpool,230,Forward,7.1,6.64,59,Man City (H),0.315,0.193,0.447,0.260,0.411
66,110,Wissa,Brentford,50,Forward,6.1,5.95,83,Leicester (H),0.281,0.076,0.336,0.345,0.359
227,401,Isak,Newcastle,206,Forward,8.5,5.95,90,Crystal Palace (A),0.320,0.136,0.413,0.268,0.424
299,541,Cunha,Wolves,30,Forward,6.9,5.88,90,Bournemouth (H),0.328,0.139,0.422,0.231,0.426
49,82,Solanke,Spurs,241,Forward,7.6,5.49,90,Fulham (H),0.391,0.093,0.448,0.308,0.524
76,129,João Pedro,Brighton,30,Forward,5.6,5.49,76,Southampton (H),0.237,0.137,0.342,0.400,0.287
96,180,N.Jackson,Chelsea,189,Forward,7.9,5.40,88,Aston Villa (H),0.334,0.159,0.440,0.304,0.431
38,58,Watkins,Aston Villa,241,Forward,9.0,5.00,85,Chelsea (A),0.285,0.161,0.400,0.148,0.364
